##### Note that for Random Forest without standaridization, the train/test periods doens't matter in the feature generating part (since there's no pca involved). We will identify them later in the code.
generate_featureset('A', 'rank', 'csv', scale = None, roll_avg = None, pca = None, shift = True, train_periods = [20162, 20171, 20172, 20181, 20182, 20191, 20192], test_periods = [20201], ties = 'min', geo = False, spatial = False)

Best params for 1st round of testing (test on 2020.1, train on 2017.1-2019.1 & val on 2019.2): 
max_depth: 3
max_features: 10
min_samples_leaf: 20
n_estimators: 30

val acc = 42.199999999999996%
test acc = 39.6%

Best params for 2st round of testing (test on 2019.2, train on 2017.1-2018.2 & val on 2019.1): 
max_depth: 3
max_features: 11
min_samples_leaf: 20
n_estimators: 30

val acc = 41.6%
test acc = 40.8%

In [1]:
# Load packages
import numpy as np
import pandas as pd
import csv
import scipy
import pickle
from CUSP_functions import generate_featureset
from CUSP_functions import evaluator

# for random forest
from sklearn.ensemble import RandomForestRegressor
# make_score is for incorportating the lc20 as gridsearch eval metric
from sklearn.metrics import r2_score,  make_scorer
# kfold is for feature selection; predefined split is for gridsearch (cuz it's faster)
from sklearn.model_selection import GridSearchCV, KFold, PredefinedSplit, cross_val_score
# pipeline is for building pipeline to feed into gridsearch
from sklearn.pipeline import Pipeline

# set display max so that you can see as many rows/cols
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

In [2]:
all_periods = [20162, 20171, 20172, 20181, 20182, 20191, 20192, 20201]

# because we will do t-1 and t-2 features to predict t overdose rank, so we start with 20171 ~ 20161 & 20162 features
first_round_train_val_periods = [20171, 20172, 20181, 20182, 20191, 20192]
first_round_test_period = [20201]

second_round_train_val_periods = [20171, 20172, 20181, 20182, 20191]
second_round_test_period = [20192]

t_minus2 = all_periods[0:-1]
t_minus1 = all_periods[1:]
t_transform = dict(zip(t_minus2, t_minus1))

In [3]:
# read in the dataset
# full = pd.read_csv("data/dataset_v4_0715_084729_A_rank____shift_min.csv")
full = generate_featureset('A', 'rank', 'csv', scale = None, roll_avg = None, pca = None, shift = True, train_periods = all_periods, test_periods = [], ties = 'min', geo = False, spatial = False)

dataset_v4_0726_164842_A_rank____shift_min.csv
total_features: 144


In [4]:
# prep the rf main df
full_rf = full.copy()

# read the same df for prep t-2 features
# in that case y of 20171 ~ x of 20162 and 20161
# so our periods for modeling would be 20171 - 20201
# in the prepared dataset, if one record is 20171, bc it's shifted, the features are already 20162's
# therefore, we need to pull 20161 features from record marked as 20162
full_2 = full.copy()
# the features pulled through this process are actually 20162 - 20191
full_t_previous = full_2[full_2.full_period.isin(t_minus2)]
# add suffix for those features to distinguish them
full_t_previous = full_t_previous.add_suffix('_t-2')
# only keep from geoid_t-1 to overdose_rank_t-1 - actually this drops the year_t-1 and period_t-1 columns
full_t_previous = full_t_previous.loc[:, 'geoid_t-2':'overdose_rank_t-2']
# rename the geoid and full_period column, and we will join the t-1 df to main df using those two columns as keys
full_t_previous = full_t_previous.rename(columns = {'geoid_t-2': 'geoid', 'full_period_t-2': 'full_period'})
# manually shift period by + 1 aka. the original record marked as 20162, which means x is from 20161, will be shifted as 20171
# in that way, the record marked as 20171 will have the features from 20161 (this is the t-2)
full_t_previous['full_period'].replace(t_transform, inplace = True)

# join to the main df
full_rf = full_rf.merge(full_t_previous, on = ['geoid', 'full_period'], how = 'left')
full_rf = full_rf[full.full_period.isin(t_minus1)]

In [17]:
# set y variable as the normalized rank
full_rf['y'] = full_rf['overdose_rank']
# create index based on period and geoid
full_rf['period_geoid'] = full_rf['full_period'].astype(str) + '_' +  full_rf['geoid'].astype(str)
# this is going to be passed into the custom scorer as the "signal" - period and geoid
full_rf.set_index('period_geoid', inplace = True)
# to make sure each period's record stay together for the kfold
full_rf.sort_index(ascending = True, inplace = True)

In [6]:
pipeline = Pipeline([('RandomForest', RandomForestRegressor(random_state = 1234))])
# put all the parameters that you would like to go through in gridsearch 
parameters = {'RandomForest__max_features': ['auto', 10, 11],
              'RandomForest__max_depth': range(3, 8),
              'RandomForest__min_samples_leaf': [20, 50, 100],
              'RandomForest__n_estimators': [30, 40, 50]}


# here it's the custimzed scorer - we need to wrap it up for feeding into gridsearch
def lc20_scorer(y_true, y_pred): 
    # y, y_pred
    # pull the period from the validation set
    period = int(list(y_true.index)[0][0:5])
    # pull geoids from the validation set
    geoid = [int(idx[6:]) for idx in list(y_true.index)]
     # call the eval function here to get the LC 20% capture
    result, rest = evaluator(np.array(y_pred), np.array(geoid), period, target_var = 'rank', ties = 'min', simple = True)
    return result.loc['20%', 'LC']

lc20 = make_scorer(lc20_scorer, greater_is_better = True)

In [7]:
include = ['PDMP_total_persons_receiving_buprenorphine_at_least_7_days',
'ACS_units_occupied_renter_pct_t-2',
'EMS_total_count_t-2',
'ACS_pop_hisp_pct',
'PDMP_total_days_supply_buprenorphine_t-2',
'PDMP_total_persons_receiving_buprenorphine_at_least_180_days',
'EMS_total_count',
'ACS_pop_hisp_white_alone_pct',
'PDMP_total_mme_dispensed',
'ACS_hh_med_income_t-2',
'ACS_pop_hisp_pct_t-2',
'PDMP_total_persons_initiating_buprenorphine_t-2',
'ACS_pop_hisp_other_alone_pct_t-2',
'segregation_classification_Predominantly Non-White_t-2',
'ACS_pop_hisp_other_alone_pct',
'PDMP_total_patients',
'PDMP_total_persons_with_multiple_prescribers_or_dispensers_t-2',
'ACS_pop_density_t-2',
'PDMP_total_persons_initiating_buprenorphine',
'ACS_pop_never_married_pct',
'ACS_pop_inc_poverty_line_ratio_0_2',
'ACS_hh_med_income',
'EMS_age_35_44_t-2',
'ACS_pop_hisp_white_alone_pct_t-2',
'ACS_pop_inc_poverty_line_ratio_2_over_t-2']

In [8]:
# here it's looking through the top k features, and for each feature set, we do gridsearch twice to test on 20201 and 20192
# we will use the feature set that has the best performances on average, and parameters accordingly

# get the list of top k featues 
#include = include
# first round of test: test 20201, train on 20162-20191 & val on 20192
# preparing the train and val data frame for gridsearch
X_train_val = full_rf[full_rf.full_period.isin(first_round_train_val_periods)].loc[:, full_rf.columns.isin(include)]
y_train_val = full_rf[full_rf.full_period.isin(first_round_train_val_periods)].loc[:, 'y']

# preparing the test data frame
X_test = full_rf[full_rf.full_period.isin(first_round_test_period)].loc[:,  full_rf.columns.isin(include)]
y_test_geoid = full_rf[full_rf.full_period.isin(first_round_test_period)].loc[:, 'geoid']
y_test = full_rf[full_rf.full_period.isin(first_round_test_period)].loc[:, 'y']

# use the predefined split to set 20171 - 20191 as training set (-1) and 20192 as validation set (1)
split_index = [-1 if int(x[0:5]) < first_round_train_val_periods[-1] else 1 for x in X_train_val.index]
cv_pd = PredefinedSplit(split_index)
# gridsearch with lc20 as the scorer
grid_search = GridSearchCV(pipeline, parameters, verbose = 2,  cv = cv_pd, n_jobs = 8, scoring = lc20)
# get the best model using training with 20171 - 20191 and validating on 20192
grid_search.fit(X_train_val, y_train_val)
# predict on 20201
y_pred = grid_search.predict(X_test)
# get the lc20 eval on 20201
test_result_df, other = evaluator(np.array(y_pred), np.array(y_test_geoid), first_round_test_period[0], target_var = 'rank', ties = 'min', simple = True)
test_result = test_result_df.loc['20%', 'LC']
# print the result
print("first test: " +  str(grid_search.best_score_) + ", " + str(test_result) +  ", params:" + str(grid_search.best_params_))

#second round of test: test 20192, train on 20162-20182 & val on 20191
# preparing the train and val data frame for gridsearch
X_train_val = full_rf[full_rf.full_period.isin(second_round_train_val_periods)].loc[:, full_rf.columns.isin(include)]
y_train_val = full_rf[full_rf.full_period.isin(second_round_train_val_periods)].loc[:, 'y']
# preparing the test data frame for gridsearch
X_test = full_rf[full_rf.full_period.isin(second_round_test_period)].loc[:,  full_rf.columns.isin(include)]
y_test_geoid = full_rf[full_rf.full_period.isin(second_round_test_period)].loc[:, 'geoid']
y_test = full_rf[full_rf.full_period.isin(second_round_test_period)].loc[:, 'y']

# use the predefined split to set 20171 - 20182 as training set (-1) and 20191 as validation set (1)
split_index = [-1 if int(x[0:5]) < second_round_train_val_periods[-1] else 1 for x in X_train_val.index]
cv_pd = PredefinedSplit(split_index)

# gridsearch with lc20 as the scorer
grid_search = GridSearchCV(pipeline, parameters, verbose = 2,  cv = cv_pd, n_jobs = 8, scoring = lc20)

# get the best model using training with 20171 - 20182 and validating on 20191
grid_search.fit(X_train_val, y_train_val)
# predict on 20192
y_pred = grid_search.predict(X_test)
# get the lc20 eval on 20192
test_result_2_df, other = evaluator(np.array(y_pred), np.array(y_test_geoid), second_round_test_period[0], target_var = 'rank', ties = 'min', simple = True)
test_result_2 = test_result_2_df.loc['20%', 'LC']
# print the result
print("second test: " +   str(grid_search.best_score_) + ", " + str(test_result_2) +  ", params:" + str(grid_search.best_params_))
print("avg test = " + str((test_result + test_result_2)/2))

Fitting 1 folds for each of 135 candidates, totalling 135 fits
0.048529517484463436
first test: 42.199999999999996, 39.6, params:{'RandomForest__max_depth': 3, 'RandomForest__max_features': 10, 'RandomForest__min_samples_leaf': 20, 'RandomForest__n_estimators': 30}
Fitting 1 folds for each of 135 candidates, totalling 135 fits
0.051151554217602646
second test: 41.6, 40.8, params:{'RandomForest__max_depth': 3, 'RandomForest__max_features': 11, 'RandomForest__min_samples_leaf': 20, 'RandomForest__n_estimators': 30}
avg test = 40.2


In [9]:
# this is the first round of test - best model

X_train_val = full_rf[full_rf.full_period.isin(first_round_train_val_periods)].loc[:, full_rf.columns.isin(include)]
y_train_val = full_rf[full_rf.full_period.isin(first_round_train_val_periods)].loc[:, 'y']

# preparing the test data frame
X_test = full_rf[full_rf.full_period.isin(first_round_test_period)].loc[:,  full_rf.columns.isin(include)]
y_test_geoid = full_rf[full_rf.full_period.isin(first_round_test_period)].loc[:, 'geoid']
y_test = full_rf[full_rf.full_period.isin(first_round_test_period)].loc[:, 'y']

reg = RandomForestRegressor(max_depth=3, max_features=10,
                     min_samples_leaf=20, n_estimators=30,
                     random_state=1234)
reg.fit(X_train_val, y_train_val)

y_pred = reg.predict(X_test)

test_result_df, other = evaluator(np.array(y_pred), np.array(y_test_geoid), first_round_test_period[0], target_var = 'rank', ties = 'min', simple = True)
test_result = test_result_df.loc['20%', 'LC']

test_result

0.048529517484463436


39.6

In [10]:
# this is the second round of test - best model

X_train_val = full_rf[full_rf.full_period.isin(second_round_train_val_periods)].loc[:, full_rf.columns.isin(include)]
y_train_val = full_rf[full_rf.full_period.isin(second_round_train_val_periods)].loc[:, 'y']

# preparing the test data frame
X_test = full_rf[full_rf.full_period.isin(second_round_test_period)].loc[:,  full_rf.columns.isin(include)]
y_test_geoid = full_rf[full_rf.full_period.isin(second_round_test_period)].loc[:, 'geoid']
y_test = full_rf[full_rf.full_period.isin(second_round_test_period)].loc[:, 'y']

reg = RandomForestRegressor(max_depth=3, max_features=11,
                     min_samples_leaf=20, n_estimators=30,
                     random_state=1234)
reg.fit(X_train_val, y_train_val)

y_pred = reg.predict(X_test)

test_result_df, other = evaluator(np.array(y_pred), np.array(y_test_geoid), second_round_test_period[0], target_var = 'rank', ties = 'min', simple = True)
test_result = test_result_df.loc['20%', 'LC']

test_result

0.051151554217602646


40.8

## Appendix: Finding best feature set

Please note that, for random forest, the model is not as stable as we would expect. The following process shows the work flow for finding the historical feature importances using the 2017.1-2018.2 data, and use top k most important features for GridSearch. The LC20% capture fluctuates 1-5% with one additional feature than the prior one. In addition, please note that when using LC20% for tuning, there could be ties in the result. The GridSearchCV function would pick up the one with the smallest index as the best parameter set.

In [11]:
# this part is for feature importance ranking - use the train set of the 2nd round of test, dropping the validation period
feature_importance_train_val_period = second_round_train_val_periods[0:-1]

# list the columns that you will not use in the X_train
not_include = ['year', 'period', 'geoid', 'full_period', 'period_geoid', 'overdose_rank', 'overdose_count','label', 'y', 'x_coord', 'y_coord']

X_train_val = full_rf[full_rf.full_period.isin(feature_importance_train_val_period)].loc[:, ~full_rf.columns.isin(not_include)]
y_train_val = full_rf[full_rf.full_period.isin(feature_importance_train_val_period)].loc[:, 'y']

# setup kfold - no shuffle because we need records of one entire period to be fed into the lc20 scorer
cv = KFold(n_splits = len(feature_importance_train_val_period), shuffle = False)

grid_search = GridSearchCV(pipeline, parameters, verbose = 2,  cv = cv, n_jobs = 8, scoring = lc20)
grid_search.fit(X_train_val, y_train_val)

# check the best params
params = list(grid_search.best_params_.values())
print(str(grid_search.best_params_))

reg = RandomForestRegressor(max_depth=params[0], max_features=params[1],
                     min_samples_leaf=params[2], n_estimators=params[3],
                     random_state=1234)
reg.fit(X_train_val, y_train_val)

# print the feature importances by descending order
feature_importance_dictionary = dict(zip(list(X_train_val.columns), list(reg.feature_importances_)))
highest_features = dict(sorted(feature_importance_dictionary.items(), key = lambda item:item[1], reverse = True))

Fitting 4 folds for each of 135 candidates, totalling 540 fits
{'RandomForest__max_depth': 4, 'RandomForest__max_features': 11, 'RandomForest__min_samples_leaf': 20, 'RandomForest__n_estimators': 50}


In [12]:
result = {}
best_parameters = {}

In [13]:
# here it's looking through the top k features, and for each feature set, we do gridsearch twice to test on 20201 and 20192
# we will use the feature set that has the best performances on average, and parameters accordingly
for k in range(12, 100):
    # get the list of top k featues 
    include = list(highest_features.keys())[0:k]
    # first round of test: test 20201, train on 20162-20191 & val on 20192    
    # preparing the train and val data frame for gridsearch
    X_train_val = full_rf[full_rf.full_period.isin(first_round_train_val_periods)].loc[:, full_rf.columns.isin(include)]
    y_train_val = full_rf[full_rf.full_period.isin(first_round_train_val_periods)].loc[:, 'y']
    
    # preparing the test data frame
    X_test = full_rf[full_rf.full_period.isin(first_round_test_period)].loc[:,  full_rf.columns.isin(include)]
    y_test_geoid = full_rf[full_rf.full_period.isin(first_round_test_period)].loc[:, 'geoid']
    y_test = full_rf[full_rf.full_period.isin(first_round_test_period)].loc[:, 'y']
    
    # use the predefined split to set 20171 - 20191 as training set (-1) and 20192 as validation set (1)
    split_index = [-1 if int(x[0:5]) < first_round_train_val_periods[-1] else 1 for x in X_train_val.index]
    cv_pd = PredefinedSplit(split_index)
    # gridsearch with lc20 as the scorer
    grid_search = GridSearchCV(pipeline, parameters, verbose = 2,  cv = cv_pd, n_jobs = 8, scoring = lc20)
    # get the best model using training with 20171 - 20191 and validating on 20192
    grid_search.fit(X_train_val, y_train_val)
    # predict on 20201
    y_pred = grid_search.predict(X_test)
    # get the lc20 eval on 20201
    test_result_df, other = evaluator(np.array(y_pred), np.array(y_test_geoid), first_round_test_period[0], target_var = 'rank', ties = 'min', simple = True)
    test_result = test_result_df.loc['20%', 'LC']
    best_params = str(grid_search.best_params_)
    # print the result
    print("first train " + str(k) + ": "+  str(grid_search.best_score_) + ", " + str(test_result) +  ", params:" + best_params)
    
    #second round of test: test 20192, train on 20162-20182 & val on 20191
    # preparing the train and val data frame for gridsearch
    X_train_val = full_rf[full_rf.full_period.isin(second_round_train_val_periods)].loc[:, full_rf.columns.isin(include)]
    y_train_val = full_rf[full_rf.full_period.isin(second_round_train_val_periods)].loc[:, 'y']
    # preparing the test data frame for gridsearch
    X_test = full_rf[full_rf.full_period.isin(second_round_test_period)].loc[:,  full_rf.columns.isin(include)]
    y_test_geoid = full_rf[full_rf.full_period.isin(second_round_test_period)].loc[:, 'geoid']
    y_test = full_rf[full_rf.full_period.isin(second_round_test_period)].loc[:, 'y']
    
    # use the predefined split to set 20171 - 20182 as training set (-1) and 20191 as validation set (1)
    split_index = [-1 if int(x[0:5]) < second_round_train_val_periods[-1] else 1 for x in X_train_val.index]
    cv_pd = PredefinedSplit(split_index)
    
    # gridsearch with lc20 as the scorer
    grid_search = GridSearchCV(pipeline, parameters, verbose = 2,  cv = cv_pd, n_jobs = 8, scoring = lc20)

    # get the best model using training with 20171 - 20182 and validating on 20191
    grid_search.fit(X_train_val, y_train_val)
    # predict on 20192
    y_pred = grid_search.predict(X_test)
    # get the lc20 eval on 20192
    test_result_2_df, other = evaluator(np.array(y_pred), np.array(y_test_geoid), second_round_test_period[0], target_var = 'rank', ties = 'min', simple = True)
    test_result_2 = test_result_2_df.loc['20%', 'LC']
    best_params_2 = str(grid_search.best_params_)
    # print the result
    print("second train " + str(k) + ": "+  str(grid_search.best_score_) + ", " + str(test_result_2) +  ", params:" + best_params_2)
    print("avg test = " + str((test_result + test_result_2)/2))
    
    # save the avg test result and best pramas of two rounds for review
    result[k] = (test_result + test_result_2)/2
    best_parameters[k] = "First round best params: " + best_params + "; Second round best params: " + best_params_2

Fitting 1 folds for each of 135 candidates, totalling 135 fits
0.04595970126790039
first train 12: 41.5, 39.6, params:{'RandomForest__max_depth': 3, 'RandomForest__max_features': 'auto', 'RandomForest__min_samples_leaf': 20, 'RandomForest__n_estimators': 30}
Fitting 1 folds for each of 135 candidates, totalling 135 fits
0.05468725750964132
second train 12: 45.0, 38.1, params:{'RandomForest__max_depth': 7, 'RandomForest__max_features': 10, 'RandomForest__min_samples_leaf': 20, 'RandomForest__n_estimators': 50}
avg test = 38.85
Fitting 1 folds for each of 135 candidates, totalling 135 fits
0.04779098994628772
first train 13: 42.199999999999996, 38.5, params:{'RandomForest__max_depth': 3, 'RandomForest__max_features': 'auto', 'RandomForest__min_samples_leaf': 20, 'RandomForest__n_estimators': 50}
Fitting 1 folds for each of 135 candidates, totalling 135 fits
0.056533064529957655
second train 13: 42.3, 40.8, params:{'RandomForest__max_depth': 4, 'RandomForest__max_features': 10, 'RandomFor

0.05118859686449939
second train 27: 42.3, 35.4, params:{'RandomForest__max_depth': 3, 'RandomForest__max_features': 10, 'RandomForest__min_samples_leaf': 20, 'RandomForest__n_estimators': 40}
avg test = 37.75
Fitting 1 folds for each of 135 candidates, totalling 135 fits
0.04524473256845962
first train 28: 41.5, 39.0, params:{'RandomForest__max_depth': 3, 'RandomForest__max_features': 10, 'RandomForest__min_samples_leaf': 100, 'RandomForest__n_estimators': 30}
Fitting 1 folds for each of 135 candidates, totalling 135 fits
0.05205262926587828
second train 28: 42.3, 39.5, params:{'RandomForest__max_depth': 3, 'RandomForest__max_features': 10, 'RandomForest__min_samples_leaf': 50, 'RandomForest__n_estimators': 40}
avg test = 39.25
Fitting 1 folds for each of 135 candidates, totalling 135 fits
0.04718137275384604
first train 29: 40.1, 40.6, params:{'RandomForest__max_depth': 4, 'RandomForest__max_features': 11, 'RandomForest__min_samples_leaf': 20, 'RandomForest__n_estimators': 30}
Fittin

0.051056644517056826
first train 43: 41.5, 39.0, params:{'RandomForest__max_depth': 4, 'RandomForest__max_features': 10, 'RandomForest__min_samples_leaf': 20, 'RandomForest__n_estimators': 30}
Fitting 1 folds for each of 135 candidates, totalling 135 fits
0.05593332562237385
second train 43: 42.3, 36.1, params:{'RandomForest__max_depth': 7, 'RandomForest__max_features': 'auto', 'RandomForest__min_samples_leaf': 20, 'RandomForest__n_estimators': 40}
avg test = 37.55
Fitting 1 folds for each of 135 candidates, totalling 135 fits
0.04143357948214532
first train 44: 38.1, 40.1, params:{'RandomForest__max_depth': 3, 'RandomForest__max_features': 11, 'RandomForest__min_samples_leaf': 50, 'RandomForest__n_estimators': 30}
Fitting 1 folds for each of 135 candidates, totalling 135 fits
0.05580829438753265
second train 44: 42.3, 36.1, params:{'RandomForest__max_depth': 7, 'RandomForest__max_features': 'auto', 'RandomForest__min_samples_leaf': 20, 'RandomForest__n_estimators': 40}
avg test = 38.1

0.05038034147881609
second train 58: 43.0, 34.0, params:{'RandomForest__max_depth': 7, 'RandomForest__max_features': 11, 'RandomForest__min_samples_leaf': 100, 'RandomForest__n_estimators': 30}
avg test = 34.4
Fitting 1 folds for each of 135 candidates, totalling 135 fits
0.04015500118404147
first train 59: 44.9, 33.7, params:{'RandomForest__max_depth': 5, 'RandomForest__max_features': 'auto', 'RandomForest__min_samples_leaf': 20, 'RandomForest__n_estimators': 30}
Fitting 1 folds for each of 135 candidates, totalling 135 fits
0.05358038054092329
second train 59: 43.0, 34.699999999999996, params:{'RandomForest__max_depth': 7, 'RandomForest__max_features': 10, 'RandomForest__min_samples_leaf': 20, 'RandomForest__n_estimators': 30}
avg test = 34.2
Fitting 1 folds for each of 135 candidates, totalling 135 fits
0.03855020557729294
first train 60: 44.2, 36.4, params:{'RandomForest__max_depth': 7, 'RandomForest__max_features': 'auto', 'RandomForest__min_samples_leaf': 20, 'RandomForest__n_est

0.044638331558651934
first train 74: 42.199999999999996, 33.2, params:{'RandomForest__max_depth': 4, 'RandomForest__max_features': 'auto', 'RandomForest__min_samples_leaf': 20, 'RandomForest__n_estimators': 50}
Fitting 1 folds for each of 135 candidates, totalling 135 fits
0.05727135872551725
second train 74: 42.3, 34.699999999999996, params:{'RandomForest__max_depth': 7, 'RandomForest__max_features': 10, 'RandomForest__min_samples_leaf': 20, 'RandomForest__n_estimators': 50}
avg test = 33.95
Fitting 1 folds for each of 135 candidates, totalling 135 fits
0.0434534645241309
first train 75: 41.5, 35.3, params:{'RandomForest__max_depth': 5, 'RandomForest__max_features': 'auto', 'RandomForest__min_samples_leaf': 20, 'RandomForest__n_estimators': 40}
Fitting 1 folds for each of 135 candidates, totalling 135 fits
0.05953770456119267
second train 75: 42.3, 36.7, params:{'RandomForest__max_depth': 7, 'RandomForest__max_features': 'auto', 'RandomForest__min_samples_leaf': 20, 'RandomForest__n_e

0.04542162374143721
second train 89: 40.9, 34.0, params:{'RandomForest__max_depth': 3, 'RandomForest__max_features': 10, 'RandomForest__min_samples_leaf': 50, 'RandomForest__n_estimators': 50}
avg test = 33.85
Fitting 1 folds for each of 135 candidates, totalling 135 fits
0.04467259415574709
first train 90: 42.199999999999996, 33.7, params:{'RandomForest__max_depth': 4, 'RandomForest__max_features': 'auto', 'RandomForest__min_samples_leaf': 20, 'RandomForest__n_estimators': 40}
Fitting 1 folds for each of 135 candidates, totalling 135 fits
0.052085954264046386
second train 90: 41.6, 34.699999999999996, params:{'RandomForest__max_depth': 5, 'RandomForest__max_features': 10, 'RandomForest__min_samples_leaf': 20, 'RandomForest__n_estimators': 40}
avg test = 34.2
Fitting 1 folds for each of 135 candidates, totalling 135 fits
0.04432858065062495
first train 91: 40.8, 35.3, params:{'RandomForest__max_depth': 4, 'RandomForest__max_features': 'auto', 'RandomForest__min_samples_leaf': 20, 'Rand

In [14]:
highest_test = dict(sorted(result.items(), key = lambda item:item[1], reverse = True))
print("The highest test accuracy comes from " + str(list(highest_test.keys())[0]) + " features.")

The highest test accuracy comes from 25 features.


In [15]:
# the four values are for: max_depth, max_features, min_samples_leaf, n_estimators, accordingly
print("Best parameters for two rounds are - " + best_parameters[list(highest_test.keys())[0]])

Best parameters for two rounds are - First round best params: {'RandomForest__max_depth': 3, 'RandomForest__max_features': 10, 'RandomForest__min_samples_leaf': 20, 'RandomForest__n_estimators': 30}; Second round best params: {'RandomForest__max_depth': 3, 'RandomForest__max_features': 11, 'RandomForest__min_samples_leaf': 20, 'RandomForest__n_estimators': 30}


In [16]:
# features that are used in the top k best performance
include[0:list(highest_test.keys())[0]]

['PDMP_total_persons_receiving_buprenorphine_at_least_7_days',
 'ACS_units_occupied_renter_pct_t-2',
 'EMS_total_count_t-2',
 'ACS_pop_hisp_pct',
 'PDMP_total_days_supply_buprenorphine_t-2',
 'PDMP_total_persons_receiving_buprenorphine_at_least_180_days',
 'EMS_total_count',
 'ACS_pop_hisp_white_alone_pct',
 'PDMP_total_mme_dispensed',
 'ACS_hh_med_income_t-2',
 'ACS_pop_hisp_pct_t-2',
 'PDMP_total_persons_initiating_buprenorphine_t-2',
 'ACS_pop_hisp_other_alone_pct_t-2',
 'segregation_classification_Predominantly Non-White_t-2',
 'ACS_pop_hisp_other_alone_pct',
 'PDMP_total_patients',
 'PDMP_total_persons_with_multiple_prescribers_or_dispensers_t-2',
 'ACS_pop_density_t-2',
 'PDMP_total_persons_initiating_buprenorphine',
 'ACS_pop_never_married_pct',
 'ACS_pop_inc_poverty_line_ratio_0_2',
 'ACS_hh_med_income',
 'EMS_age_35_44_t-2',
 'ACS_pop_hisp_white_alone_pct_t-2',
 'ACS_pop_inc_poverty_line_ratio_2_over_t-2']